THIS SCRIPT PREPROCESSES THE CONSOLIDATED DATA COLLECTED AND CREATES SEPERATE TABLES FOR ANALYSIS

In [13]:
# Dependencies
from config import homeDepot_API
from serpapi import GoogleSearch
import pandas as pd
import numpy as np
from openpyxl import load_workbook

STEP 1: PREPROCESS DATA

In [14]:
# Collect all data

path0 = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\HistoricalPrices.csv"
raw_prices = pd.read_csv(path0)

path = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\data.csv"
data = pd.read_csv(path)

print(data.shape, data.dtypes)

(1250, 6) material_description     object
material_price          float64
vendor                   object
location                 object
formatted_datetime       object
dow                      object
dtype: object


In [15]:
# Create Regional columns

# Split location column to create a zip code column

data['zip'] = (np.where(data['location'].str.contains(','), data['location'].str.split(',').str[-1], data['location']))
data['zip'] = data['zip'].str.strip()
data['zip'] = (np.where(data['zip'].str.len()>5, "other", data['zip']))

region = []
for i in data['zip']:
    if i.startswith("0"):
        region.append('northeast')
    elif i.startswith("1"):
        region.append('northeast')
    elif i.startswith("2"):
        region.append('east')
    elif i.startswith("3"):
        region.append('southeast')
    elif i.startswith("4"):
        region.append('midwest')
    elif i.startswith("5"):
        region.append('northplaines')
    elif i.startswith("6"):
        region.append('central')
    elif i.startswith("7"):
        region.append('south')
    elif i.startswith("8"):
        region.append('mountains')
    elif i.startswith("9"):
        region.append('west')
    else:
        region.append('other')
        
        
data['region'] = region
data.formatted_datetime = pd.to_datetime(data.formatted_datetime)
data['date'] = data.formatted_datetime.dt.date
data.date = pd.to_datetime(data.date)
data.drop(columns =['formatted_datetime'], inplace = True)

In [16]:
raw = []
for i in data['material_description']:
    if i.startswith("1") or i.startswith("3"):
        raw.append('steel')    
    else:
        raw.append('copper')
        
        
data['raw_material'] = raw

In [17]:
data.dtypes

material_description            object
material_price                 float64
vendor                          object
location                        object
dow                             object
zip                             object
region                          object
date                    datetime64[ns]
raw_material                    object
dtype: object

In [18]:
# Create raw prices features (NOTE: SOME HOW, I LOSE 100 PLUS ROWS OF DATA??)

raw_prices.date = pd.to_datetime(raw_prices.date)
raw_prices = raw_prices.iloc[:,[0,1,2]]
data = pd.merge(data, raw_prices)
data.shape

(1161, 11)

In [ ]:
# Creating 'material_description' by extracting from Home Depot. Home Depot's product id is what is used to query

material_name = []

ids = ['100400409', '100400406', '100137321', '100400405', 
       '202241090', '100144234', '202077145', '202077147', 
       '202288508', '202304774', '202304649', '202304645', 
       '202019377', '202316275', '202019375', '202316276']

for i in ids:

    params = {
      "engine": "home_depot_product",
      "product_id": i,
      "api_key": homeDepot_API
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    products = results['product_results']
    
    material_name.append(products['title'])
        
material_description = pd.DataFrame(data={'material_description': material_name}).iloc[[3,1,0,5,8,4,2,6,7,12,13,14,15,9,10,11],:].reset_index(drop=True)


https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


In [ ]:
R = material_description['material_description']
R_list=R.tolist()
target_list = [[]]
for r in R_list:
    int_df = data.loc[data['material_description']==r] [['material_price']]
    Q1 = np.quantile(int_df, 0.25)
    Q3 = np.quantile(int_df, 0.75)
    data.loc[data['material_description']==r, 'target_price'] = np.where(int_df['material_price'] >= Q3, 'high price', (np.where(int_df['material_price'] <= Q1, 'low price', 'about average')))

data.head()

In [ ]:
descriptive_table = data.groupby('material_description').describe()['material_price'].transpose()
descriptive_table = descriptive_table.iloc[:,[5,12,2,3,11,0,4,10,1,8,9,6,7,14,15,13]]
descriptive_table.columns=mat_name = ['emt12','emt34','emt1','cup12','cup34','cup1','con12','con34','con1','rom142','rom143','rom122','rom123','wir4','wir6','wir10']
descriptive_table

In [ ]:
# Creating a dataframe for each material item
new_df = []
for r in R:
    df = data[data['material_description']==r].reset_index(drop=True)    
    new_df.append(df)
test = pd.concat(new_df, axis=0, ignore_index=True)

In [ ]:
preprocessed_df = data
preprocessed_df.shape

In [ ]:
# Send CSVs to my computer

preprocessed = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\preprocessed_df.csv"
preprocessed_df.to_csv(preprocessed, index=False)

descriptive = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\descriptive_table.csv"
descriptive_table.to_csv(descriptive, index=False)

emt12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\emt12.csv"
new_df[0].to_csv(emt12_df, index=False)

emt34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\emt34.csv"
new_df[1].to_csv(emt34_df, index=False)

emt1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\emt1.csv"
new_df[2].to_csv(emt1_df, index=False)

cup12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\cup12.csv"
new_df[3].to_csv(cup12_df, index=False)

cup34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\cup34.csv"
new_df[4].to_csv(cup34_df, index=False)

cup1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\cup1.csv"
new_df[5].to_csv(cup1_df, index=False)

con12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\con12.csv"
new_df[6].to_csv(con12_df, index=False)

con34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\con34.csv"
new_df[7].to_csv(con34_df, index=False)

con1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\con1.csv"
new_df[8].to_csv(con1_df, index=False)

rom142_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom142.csv"
new_df[9].to_csv(rom142_df, index=False)

rom143_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom143.csv"
new_df[10].to_csv(rom143_df, index=False)

rom122_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom122.csv"
new_df[11].to_csv(rom122_df, index=False)

rom123_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom123.csv"
new_df[12].to_csv(rom123_df, index=False)

wir4_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\wir4.csv"
new_df[13].to_csv(wir4_df, index=False)

wir6_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\wir6.csv"
new_df[14].to_csv(wir6_df, index=False)

wir10_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\wir10.csv"
new_df[15].to_csv(wir10_df, index=False)